### General training pipeline in pytorch:
1) Design model (input, output size, forward pass )

2) Construct loss and optimizer

3) Training loop:

    - Forward pass: Compute prediction
    
    - Backward pass: Gradients
    
    - Update weights

In [4]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## 0) Prepare data

In [5]:
bc = datasets.load_breast_cancer()

X, y = bc.data, bc.target

n_samples, n_features = X.shape

print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

569 30


### Scale

In [6]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshaping tensor...
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

---

## 1) Model

### Logistic regression

f = wx + b, .....with a sigmoid function at the end

In [7]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)  # 1 is the output size...
        
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))   # will return a value between 0 and 1, above 0.05 , its 1...
        return y_predicted
    

model = LogisticRegression(n_features)

---

## 2) Loss and Optimizer

In [8]:
learning_rate = 0.01
criterion = nn.BCELoss()  # Binary cross entropy loss...
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

---

## 3) Training Loop

In [10]:
num_epochs = 100

for epoch in range(num_epochs):
    
    # Forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    # Backward pass
    loss.backward()
    
    # Updates
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
# Evaluation.....this should not be a part of the computational graph...
with torch.no_grad():
    y_predicted = model(X_test)
    
    y_predicted_cls = y_predicted.round()
    
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    
    print(f'Accuracy = {acc:.4f}')

epoch: 10, loss = 0.5094
epoch: 20, loss = 0.4399
epoch: 30, loss = 0.3912
epoch: 40, loss = 0.3552
epoch: 50, loss = 0.3274
epoch: 60, loss = 0.3052
epoch: 70, loss = 0.2870
epoch: 80, loss = 0.2718
epoch: 90, loss = 0.2588
epoch: 100, loss = 0.2475
Accuracy = 0.8684


---